<a href="https://colab.research.google.com/github/Flantropy/notebooks/blob/main/TF_intro_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiclass classification with **fashion-MNIST**

In [ ]:
#@title Imports { display-mode: "code" }
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import random
import itertools
from google.colab import files
from collections import namedtuple
from operator import attrgetter

## Hands on data

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
# normalizing data
X_train_norm = X_train / 255.0
X_test_norm = X_test / 255.0

In [ ]:
# one-hot encoding training and test labels
y_train_one_hot = tf.one_hot(y_train, 10)
y_test_one_hot = tf.one_hot(y_test, 10)

In [ ]:
clothes_names = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

In [ ]:
def plot_item(index, scope=X_train, label_scope=y_train):
    plt.imshow(scope[index], cmap=plt.cm.binary)
    plt.title(clothes_names[label_scope[index]])

In [ ]:
plt.figure(figsize=(7, 7))
for i in range(4):
    ax = plt.subplot(2, 2, i+1)
    index = random.randrange(0, int(6e+4))
    plot_item(index)
    plt.axis(False)

## Model

In [ ]:
# check gpu device
tf.test.gpu_device_name()

In [ ]:
#@title Letto { display-mode: "code" }
tf.random.set_seed(42)
layers = [
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
]

letto = tf.keras.Sequential(layers)
letto.compile(
    tf.keras.optimizers.Adam(),
    tf.keras.losses.CategoricalCrossentropy(),
    ['accuracy'],
)

history = letto.fit(
    X_train_norm,
    y_train_one_hot,
    epochs=20,
    # validation_data=(X_test_norm, y_test_one_hot),
)

In [ ]:
#@title Loading Model { display-mode: "code" }
# Copy model from Drive
#!cp /content/drive/MyDrive/ColabNotebooks/TF/models/letto_A.h5 /content
# Loading model
#letto = tf.keras.models.load_model('/content/letto_A.h5')

In [ ]:
letto.evaluate(X_test_norm, y_test_one_hot)

In [ ]:
pd.DataFrame(history.history).plot()

In [ ]:
pd.DataFrame(history.history['loss']).plot()

In [ ]:
letto.summary()

### Saving model

In [ ]:
#letto.save('letto_A.h5')
#!cp /content/letto_A.h5 /content/drive/MyDrive/ColabNotebooks/TF/models

In [ ]:
# Finding learning rate
# lrs = 1e-3 * (10**(tf.range(40)/20))
# plt.semilogx(lrs, find_lr_history.history["loss"])

## Creating Confusion Matrix

In [ ]:
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(15, 15), text_size=10):
  # Create the confusion matrix
  cm = sk.metrics.confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize our confusion matrix
  n_classes = cm.shape[0]

  # Let's prettify it
  fig, ax = plt.subplots(figsize=figsize)
  # Create a matrix plot
  cax = ax.matshow(cm, cmap=plt.cm.Blues)
  fig.colorbar(cax)

  # Set labels to be classes 
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])

  # Label the axes
  ax.set(title="Confusion Matrix",
        xlabel="Predicted Label",
        ylabel="True Label",
        xticks=np.arange(n_classes),
        yticks=np.arange(n_classes),
        xticklabels=labels, 
        yticklabels=labels)

  # Set x-axis labels to bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  # Adjust label size
  ax.yaxis.label.set_size(text_size)
  ax.xaxis.label.set_size(text_size)
  ax.title.set_size(text_size)

  # Set threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
            horizontalalignment="center",
            color="white" if cm[i, j] > threshold else "black",
            size=text_size)

In [ ]:
y_probs = letto.predict(X_test_norm)

In [ ]:
y_pred = y_probs.argmax(axis=1)

In [ ]:
sk.metrics.confusion_matrix(y_test, y_pred)

In [ ]:
make_confusion_matrix(y_test, y_pred, clothes_names)

In [ ]:
def plot_random_image(model: tf.keras.Model, images, true_labels, classes):
    position = random.randrange(0, len(images))
    random_image = images[position]
    pred_prob = model.predict(random_image.reshape(1, 28, 28))
    percentile = 100 * tf.reduce_max(pred_prob)
    pred_label = classes[pred_prob.argmax()]
    true_label = classes[true_labels[position]]
    color = 'green' if pred_label == true_label else 'red'
    plt.imshow(random_image, cmap=plt.cm.binary)
    plt.xlabel(f'Predicted: {pred_label} {percentile:2.0f} (True: {true_label}) ',
               color=color)


In [ ]:
plot_random_image(letto, X_test_norm, y_test, clothes_names)

## Explore our model

In [ ]:
letto.layers

In [ ]:
weigths, biases = letto.layers[1].get_weights()

In [ ]:
weigths, weigths.shape

In [ ]:
biases, biases.shape

In [ ]:
tf.keras.utils.plot_model(letto, show_shapes=True)